In [ ]:
from AFLPy.AFLData_Client import load_data

In [ ]:
chains = load_data(Dataset_Name="AFL_API_Match_Chains", ID = "AFL")

In [ ]:
import joblib
import warnings
warnings.filterwarnings('ignore')
xscore_preproc = joblib.load("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/xscore_preprocessor.joblib")
xscore_model = joblib.load("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models/catboost_xscore_model.joblib")   

In [ ]:
shots_features = xscore_preproc.transform(chains)

In [ ]:
shots = chains.loc[shots_features.index]

In [ ]:
model_features = xscore_model.cb_clf.feature_names_
model_features

In [ ]:
shots['predicted_result'] = xscore_model.predict(shots_features[model_features]).flatten()

In [ ]:
shots[[f'{x}_probas' for x in xscore_model.classes_]] = xscore_model.predict_proba(shots_features[model_features])
shots['xscore'] = shots['goal_probas']*6 + shots['behind_probas']

In [ ]:
shots.shape

In [ ]:
import pandas as pd
shots = shots.merge(shots_features, left_index=True, right_index=True)
shots.shape

In [ ]:
import joblib
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

def predict_xscore_from_chains(chains):
    
    chains = chains.reset_index(drop=True)

    xscore_preproc = joblib.load("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/xscore_preprocessor.joblib")
    shots_features = xscore_preproc.transform(chains)

    xscore_model = joblib.load("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models/catboost_xscore_model.joblib")   
    model_features = xscore_model.cb_clf.feature_names_
    
    shots = chains.loc[shots_features.index]
    shots['predicted_result'] = xscore_model.predict(shots_features[model_features]).flatten()
    shots[[f'{x}_probas' for x in xscore_model.classes_]] = xscore_model.predict_proba(shots_features[model_features])
    shots['xscore'] = shots['goal_probas']*6 + shots['behind_probas']
    shots = shots.merge(shots_features, left_index=True, right_index=True)

    return shots

In [ ]:
shots = predict_xscore_from_chains(chains)

In [ ]:
match_id = 'AFL_2022_F4_Geelong_Sydney'
predict_xscore_from_chains(chains[chains['Match_ID'] == match_id])